# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


*Есть незаополненные данные в столбцах 'days_imployed' и 'total_income'. Если между этими пропусками есть связь, то скорее всего это не случайные пропуски*

In [2]:
bool_series = pd.isnull(data['days_employed'])
data[bool_series]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


**Вывод**

Да, в таблице, если не заполнено поле 'days_employed', то и поле 'total_income' имеет значение NaN. Предположим, что это люди недостаточно проработавшие на своём месте работы, чтобы получить справку о доходах. 

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
jobs = data['income_type'].unique()

In [4]:
jobs_median = {}
for job in jobs:
    jobs_median[job] = data[data['income_type'] == job]['total_income'].median()
jobs_median

{'сотрудник': 142594.39684740017,
 'пенсионер': 118514.48641164352,
 'компаньон': 172357.95096577113,
 'госслужащий': 150447.9352830068,
 'безработный': 131339.7516762103,
 'предприниматель': 499163.1449470857,
 'студент': 98201.62531401133,
 'в декрете': 53829.13072905995}

In [5]:
for job in jobs_median:
    data.loc[data['income_type'] == job, 'total_income'] = jobs_median[job]

**Вывод**

Пропусков больше 10%, поэтому заполним пропуски медианными значениями по месту работы, чтобы эти пропуски не сильно смещали зароботную плату работников. Также заполним нулями количество отработанных дней, так как нам не нужен стаж работы для проверки наших гипотез.

In [6]:
employed_median = {}
for job in jobs:
    employed_median[job] = data[data['income_type'] == job]['days_employed'].median()
jobs_median

{'сотрудник': 142594.39684740017,
 'пенсионер': 118514.48641164352,
 'компаньон': 172357.95096577113,
 'госслужащий': 150447.9352830068,
 'безработный': 131339.7516762103,
 'предприниматель': 499163.1449470857,
 'студент': 98201.62531401133,
 'в декрете': 53829.13072905995}

In [7]:
for job in employed_median:
    data.loc[data['income_type'] == job, 'days_employed'] = employed_median[job]

In [8]:
years_employed = []
for job in jobs_median:
    data.loc[data['income_type'] == job, 'total_income'] = jobs_median[job]
for i in range(len(data)):
    if data.loc[i, 'days_employed'] >= 0:
        years_employed.append(data.loc[i, 'days_employed'] / 365 / 24)
    else:
        years_employed.append(abs((data.loc[i, 'days_employed'] / 365)))   
data['years_employed'] = years_employed

In [9]:
family_status = data['family_status'].unique()
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [10]:
average_dob = {}
for status in family_status:
    average_dob[status] = data[data['family_status'] == status]['dob_years'].median()
for i in range(len(data)):
    for status in average_dob:
        if (data.loc[i, 'family_status'] == status) and (data.loc[i, 'dob_years'] == 0):
            data.loc[i, 'dob_years'] = average_dob[status]

In [11]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [12]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
10701,0,-1547.382223,24.0,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,172357.950966,покупка недвижимости,4.239403


In [13]:
data = data.drop(index = [10701])

### Замена типа данных

In [14]:
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)

**Вывод**

Поменяем типы данных для столбцов 'days_employed' и 'total_income' для более простых вычислений, а также количество дней бывает только целым.

### Обработка дубликатов

In [15]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender'] = data['gender'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()
data = data.drop_duplicates().reset_index(drop = True)

**Вывод**

Удаляем все полные дубликаты и переводим все строки в нижний регистр для отслеживания дубликатов, отличающихся только регистром. Как оказалось у нас целых 3642 строки являются полными дубликатами. Используется метод drop_duplicates() без параметров, что приводит к уничтожению всех полностью дублирующихся строк. Дубликаты могли появится из за нескольких запросов на получение кредита от одного и того-же человека, а также возможно, что эта создана из нескольких таблиц с пересекающимися данными, так как дубликатов очень много.

### Лемматизация

In [16]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
purposes = data['purpose'].unique()
purposes_lemmas_column = []
purposes_lemmas = []
for i in range(len(purposes)):
    purposes_lemmas_column.append(m.lemmatize(purposes[i]))
#Далее создаём список лемм, с условием уникальности этих лемм
for lemmas in purposes_lemmas_column: 
    for lemma in lemmas:
        if (lemma not in purposes_lemmas) and (len(lemma) > 2):
            purposes_lemmas.append(lemma)
purposes_lemmas

['покупка',
 'жилье',
 'приобретение',
 'автомобиль',
 'дополнительный',
 'образование',
 'сыграть',
 'свадьба',
 'операция',
 'проведение',
 'для',
 'семья',
 'недвижимость',
 'коммерческий',
 'жилой',
 'строительство',
 'собственный',
 'подержать',
 'свой',
 'заниматься',
 'сделка',
 'подержанный',
 'получение',
 'высокий',
 'профильный',
 'сдача',
 'ремонт']

Мы получили небольшой список лемм, которые используются при формулировке цели. Здесь мы уже в ручную можем разделить какие леммы отвечают за какие-то категории, а какие нет. А также разделить леммы на категегории по цели креитов, к которым они относятся, так например леммы 'жилье' и 'недвижимость' относятся к одной категории - операции с жильём.

In [17]:
#Создаём список нужных нам целей
purposes_lemmas_res = {'house':[purposes_lemmas[1],purposes_lemmas[12]], 
                       'education': [purposes_lemmas[5]],
                      'car_loan': [purposes_lemmas[3]],
                      'wedding': [purposes_lemmas[7]]}

lemmas_temp = []
#Создаём лемматизированный столбец, для дальнейшей категоризации
for i in range(len(data)):
    lemmas_temp.append(m.lemmatize(data.loc[i, 'purpose']))
lemmas_purpose = []
#Находим ключевые леммы и если находим лемму из списка, соответствующего категории добавляем категорию в столбец
for lemmas in lemmas_temp:
    for lemma in lemmas:
        for purposes in purposes_lemmas_res:
            for purpose in purposes_lemmas_res[purposes]:
                if lemma == purpose:
                    lemmas_purpose.append(purposes)
                    break
#На выходе получили столбец, который даёт нам цель кредита в удобной для обработки форме
data['lemma_purpose'] = lemmas_purpose

**Вывод**

При помощи лемматизации мы нашли 4 основных цели получения кредита и леммы для них, а также добавили столбец в изначальную таблицу с этими целями, что сводит последующую категоризацию данных к простой логической индексации.

### Категоризация данных

Для начала найдём средний "индекс задолженности", чтобы знать, как часто люди не отдают кредиты вовремя в среднем. 

In [18]:
average_index = data['debt'].mean()
average_index

0.09523542914730418

Далее при помощи логической индексации и метода mean() найдём также индексы для разных категорий людей.

In [19]:
with_child_index = data[data['children'] != 0]['debt'].mean()
without_child_index = data[data['children'] == 0]['debt'].mean()
print('С детьми:', with_child_index)
print('Без детей:', without_child_index)

С детьми: 0.1000754716981132
Без детей: 0.09238163047347811


In [20]:
family_status = {}
keys = data['family_status'].unique()
family_status[keys[0]] = data[data['family_status'] == keys[0]]['debt'].mean()
family_status[keys[1]] = data[data['family_status'] == keys[1]]['debt'].mean()
family_status[keys[2]] = data[data['family_status'] == keys[2]]['debt'].mean()
family_status[keys[3]] = data[data['family_status'] == keys[3]]['debt'].mean()
family_status[keys[4]] = data[data['family_status'] == keys[4]]['debt'].mean()
family_status

{'женат / замужем': 0.0917625681405209,
 'гражданский брак': 0.11338345864661654,
 'вдовец / вдова': 0.07482185273159145,
 'в разводе': 0.0738488271068636,
 'не женат / не замужем': 0.1012514220705347}

In [21]:
average_income = data['total_income'].median()
average_income

142594.0

In [22]:
income_index = {}
data['total_income'].describe()

count     17861.000000
mean     147162.310453
std       17855.216847
min       53829.000000
25%      142594.000000
50%      142594.000000
75%      172357.000000
max      499163.000000
Name: total_income, dtype: float64

In [23]:
data['income_index'] = pd.qcut(data['total_income'], q = 6, duplicates = 'drop') 
income_type = data['income_index'].unique()
income_type_index = {}
for income in income_type:
    income_type_index[income] = data[data['income_index'] == income]['debt'].mean()
income_type_index    

{Interval(53828.999, 142594.0, closed='right'): 0.10491637100861632,
 Interval(150447.0, 172357.0, closed='right'): 0.08075340982896731,
 Interval(142594.0, 150447.0, closed='right'): 0.06134094151212553,
 Interval(172357.0, 499163.0, closed='right'): 0.0}

In [24]:
purpose_index = {}
for purpose in purposes_lemmas_res:
    purpose_index[purpose] = data[data['lemma_purpose'] == purpose]['debt'].mean()
purpose_index

{'house': 0.08430136076999668,
 'education': 0.10338983050847457,
 'car_loan': 0.10657718120805369,
 'wedding': 0.11303789338471419}

Тут мы воспользовались уже созданным словарём лемм для нахождения "индекса задолженности" по разным целям кредитов.

**Вывод**

Как такового вывода здесь нет, так как он и является ответом на вопросы из шага 3. Возможно нужно было сделать отдельно категоризацию, а на третьем шаге применять метод mean(), но это странно.

## Шаг 3. Ответьте на вопросы

In [25]:
average_index

0.09523542914730418

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [26]:
print('С детьми:', with_child_index)
print('Без детей:', without_child_index)

С детьми: 0.1000754716981132
Без детей: 0.09238163047347811


**Вывод**

Наличие детей повышает шанс задолженности более чем на 2,6%

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
print('не женат / не замужем:', family_status['не женат / не замужем'])
print('гражданский брак:', family_status['гражданский брак'])
print('женат / замужем:', family_status['женат / замужем'])
print('в разводе:', family_status['в разводе'])
print('вдовец / вдова:', family_status['вдовец / вдова'])

не женат / не замужем: 0.1012514220705347
гражданский брак: 0.11338345864661654
женат / замужем: 0.0917625681405209
в разводе: 0.0738488271068636
вдовец / вдова: 0.07482185273159145


**Вывод**

Как мы видим семейное положение влияет на наш индекс. С каждым пройденным этапом в жизни люди всё реже не отдают кредит.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
income_type_index
#Сорри, что не запарился с нормальной выдачей интервалов :)

{Interval(53828.999, 142594.0, closed='right'): 0.10491637100861632,
 Interval(150447.0, 172357.0, closed='right'): 0.08075340982896731,
 Interval(142594.0, 150447.0, closed='right'): 0.06134094151212553,
 Interval(172357.0, 499163.0, closed='right'): 0.0}

**Вывод**

Самыми большими должниками являются самые бедные граждане, далее идёт резкое снижение индекса, люди берущие кредит и имеющие средний доход довольно редко имеют задолженности, однако люди с доходом выше среднего чаще имеют задолженности, возможно это связанно с тем, что они берут кредиты на улучшение своего материального положения, что не всегда исполняется по графику, что и создаёт задолженности. Богатые же люди практически не имеют задолженностей, так как у них уже есть средства для отдачи кредитов.

- Как разные цели кредита влияют на его возврат в срок?

In [29]:
purpose_index
print('Недвижимость:', purpose_index['house'])
print('Образование:', purpose_index['education'])
print('Автомобильный кредит:', purpose_index['car_loan'])
print('Свадьба:', purpose_index['wedding'])

Недвижимость: 0.08430136076999668
Образование: 0.10338983050847457
Автомобильный кредит: 0.10657718120805369
Свадьба: 0.11303789338471419


**Вывод**

Кредиты на недвижимость и свадьбы отдаются чаще, чем кредиты на образование и автомобили.

## Шаг 4. Общий вывод

В среднем люди довольно редко не отдают кредиты(8% случаев), также вне зависимости от целей, наличия детей или семейного положения люди не отдают кредит в 6-10% случаев. Самым надёжным критерием для банка является высокий доход клиента, так как он практически гарантирует возврат кредита в срок. Остальные критерии не могут точно оценить платёжеспособность клиента, решение об одобрении или отказе от кредита стоит нужно давать на основе суммарного вклада разных критериев.